In [322]:
import json
import logging
from datetime import datetime
from __future__ import annotations
from typing import Optional, Any, Union
from sqlglot import parse_one, Expression, column, select, condition, alias, and_
from sqlglot.expressions import Predicate, In, Join, EQ, Select, Binary, Not, Subquery
configuraciones = json.load(open("./configuraciones.json"))
OPERACIONES_CONJUNTOS = configuraciones['miniconsultas_operaciones']
STATUS = configuraciones['miniconsultas_status']
FUNCIONES_AGREGACION = configuraciones['miniconsultas_funciones_agregacion']
DEBUG = configuraciones['debug']

In [323]:
class miniconsulta_sql:
    """
        Clase que controla la ejecucion de una consulta 'simple' de SQL usando un LLM.

        Tenga en cuenta que estas consultas solo trabajan sobre una unica tabla. Si
        En la condiciones de la consulta se hace referencia a otra tabla quiere decir
        que esta consulta depende de otra, y por lo tanto el atributo dependencia 
        debe ser distinto de None

        atributos
        ------------

        tabla: Un string que almacena el nombre original de la tabla SQL.

        alias: Un string que almacena el alias relacionado a la tabla SQL.

        proyecciones: Una lista de expresiones las cuales son todas las proyecciones
                      del select de la consulta SQL.

        condiciones: Una lista de expresiones las cuales son todas las condiciones
                     del where de la consulta SQL.
                     
        condiciones_join: Una lista de expresiones las cuales son condiciones que 
                          estaban en algun ON de un JOIN en la consulta original 
                          SQL, estas condiciones deben ir en el where de esta
                          miniconsulta, e indica la forma en la que se debe
                          juntar el resultado de esta consulta con otra.
        
        status: Un string que indica el estado de ejecicion de la peticion a SQL.

        dependencia: Una miniconsulta de la cual depende esta miniconsulta.

        metodos
        ----------
        crear_prompt: Funcion que usando los datos disponibles crea una 
                      version en lenguaje natural de la peticion SQL
        
        _crear_representacion_SQL: Usando los datos disponibles
                                   devuelve un string con la miniconsulta
                                   en sintaxis SQL de Postgres
                                   
    """
    # Toda la información necesaria para construir la 
    # consulta SQL
    tabla:str
    alias: str
    proyecciones: list[Expression]
    condiciones: list[Expression]
    condiciones_join: Optional[list[Expression]]
    
    # Status disponibles: En espera, Ejecutando, Finalizado
    status: str
    dependencia: Optional[list[miniconsulta_sql]]

    def __init__(self, 
                tabla: str, 
                proyecciones: list[Expression],
                condiciones: list[Expression],
                alias:str = '',
                condiciones_join: Optional[list[Expression]] = None,
                dependencia: Optional[miniconsulta_sql] = None):
        
        self.tabla = tabla
        self.alias = alias
        self.proyecciones = proyecciones
        self.condiciones = condiciones
        self.condiciones_join = condiciones_join
        self.dependencia = dependencia
        self.status = STATUS[0]

    def crear_prompt(self):
        raise Exception("Por implementar!!!")
    
    """def _crear_representacion_SQL(self) -> str:
                
        lista_condiciones = []

        if len(self.condiciones) != 0:
            lista_condiciones += self.condiciones

        if self.condiciones_join is not None and len(self.condiciones_join) != 0:
            lista_condiciones += self.condiciones_join
        
        if len(lista_condiciones) == 0:
            raise Exception("La miniconsulta debe tener al menos una condicion de cualquier tipo")
        
        condicion = lista_condiciones[0]
        
        for otra_condicion in lista_condiciones[1:]:
            condicion = condicion.and_(otra_condicion)

        tabla_form = self.tabla
        
        if self.alias != '':
            tabla_form = alias(f'{self.tabla} AS {self.alias}',self.alias,dialect='postgres',)

        return select(*self.proyecciones).from_(tabla_form).where(condicion).sql(dialect='postgres')
    """
    def imprimir_datos(self, nivel:int) -> str:
                
        return f"""
{nivel*'    '}MINI CONSULTA
{(nivel+1)*'    '}tabla: {self.tabla}
{(nivel+1)*'    '}alias: {self.alias}
{(nivel+1)*'    '}proyecciones: {[i.sql() for i in self.proyecciones]}
{(nivel+1)*'    '}condiciones: {[i.sql() for i in self.condiciones]}
{(nivel+1)*'    '}condiciones_join: {[i.sql() for i in self.condiciones_join]}
{(nivel+1)*'    '}dependencia: {f'[{self.dependencia.imprimir_datos(nivel + 2) if self.dependencia != None else ""}]'}
{(nivel+1)*'    '}status: {self.status}
{(nivel+1)*'    '}"""

    def __str__(self) -> str:
        return self.imprimir_datos(0)
    
    def __repr__(self) -> str:
        return self.imprimir_datos(0)

In [324]:
class join_miniconsultas_sql:
    """
        Clase que controla todos los datos necesarios para realizar
        una consulta que originalmente era un join, utilizando
        miniconsultas de complejidad menor

        atributos
        ------------        
        miniconsultas_dependientes: Lista con todas las miniconsultas
                                    que dependen del resultado de otra
                                    para ser ejecutada con exito 

        miniconsultas_independientes: Lista con todas las miniconsultas
                                      que no dependen del resultado
                                      de ninguna otra para ser ejecutada
                                      con exito

        condiciones_having_or: Lista con todas las disyunciones que existen
                               en el HAVING del JOIN

        lista_agregaciones: Lista con todos las funciones de agregación que 
                            esta en el SELECT del JOIN
        
        lista_group_by: Lista con todas las columnas del GROUP BY

        lista_order_by: Lista con todas las columnas del ORDER BY

        condiciones_join: Una lista con las distintas condiciones
                          utilizadas en los joins de la consulta
                          SQL original
        
        condiciones_or: Lista con todas las disyunciones que existen
                        en el WHERE del JOIN
        
        condiciones_having: Las condiciones del HAVING que no son disyunciones
        
        resultado: El resultado de la ejecucion de este join

        limite: Un entero que indica si el JOIN tiene un LIMIT o no (Si tiene
                un -1 quiere decir que no tienen LIMIT)

        metodos
        -------------
        ejecutar: Funcion que realizara todos los joins utilizando las
                  miniconsultas disponibles
    """
    miniconsultas_independientes: list[miniconsulta_sql]
    miniconsultas_dependientes: list[miniconsulta_sql]
    condiciones_having_or: list[Expression]
    lista_agregaciones: list[Expression]
    condiciones_having: list[Expression]
    lista_group_by: list[dict[str,str]]
    lista_order_by: list[dict[str,str]]
    condiciones_join: list[Expression]
    condiciones_or: list[Expression]
    resultado: str
    limite: int

    def __init__(self, 
                 condiciones_join: list[Expression],
                 miniconsultas_dependientes: list[miniconsulta_sql],
                 miniconsultas_independientes: list[miniconsulta_sql],
                 lista_group_by: list[dict[str,str]] = [],
                 lista_order_by: list[dict[str,str]] = [],
                 limite: int = -1,
                 condiciones_or: list[Expression] = [],
                 condiciones_having_or: list[Expression] = [],
                 condiciones_having: list[Expression] = [],
                 lista_agregaciones: list[Expression] = []
                 ):
        
        self.condiciones_join = condiciones_join
        self.miniconsultas_dependientes = miniconsultas_dependientes
        self.miniconsultas_independientes = miniconsultas_independientes
        self.lista_group_by = lista_group_by
        self.lista_order_by = lista_order_by
        self.limite = limite
        self.condiciones_or = condiciones_or
        self.condiciones_having_or = condiciones_having_or
        self.condiciones_having = condiciones_having
        self.lista_agregaciones = lista_agregaciones

    def ejecutar(self):
        """
            Aqui es donde se hara el o los joins usando los resultados
            de las miniconsultas.

            Ten en cuenta que este ejecutar debe ser llamado despues de 
            haber realizado todas las peticiones al LLM y las miniconsultas
            deben haber sido ejecutadas antes de ejecutar esta funcion

            Aqui probablemente le pasemos distintas estrategias para ejecutar un join
            por lo que ten presente que seguramente debamos pasarle de alguna forma
            la manera en la que vamos a ejecutar
        """
        raise Exception("Por implementar!!!")
    
    def imprimir_datos(self, nivel: int) -> str:
        return f"""
{nivel*'    '}CONSULTA JOIN
{(nivel + 1)*'    '}miniconsultas_independientes: {self.miniconsultas_independientes}
{(nivel + 1)*'    '}miniconsultas_dependientes: {self.miniconsultas_dependientes}
{(nivel + 1)*'    '}condiciones_having_or: {[i.sql() for i in self.condiciones_having_or]}
{(nivel + 1)*'    '}condiciones_having: {[i.sql() for i in self.condiciones_having]}
{(nivel + 1)*'    '}lista_group_by: {self.lista_group_by}
{(nivel + 1)*'    '}lista_order_by: {self.lista_order_by}
{(nivel + 1)*'    '}condiciones_join: {[i.sql() for i in self.condiciones_join]}
{(nivel + 1)*'    '}condiciones_or: {[i.sql() for i in self.condiciones_or]},
{(nivel + 1)*'    '}lista_agregaciones: {[i.sql() for i in self.lista_agregaciones]}
{(nivel + 1)*'    '}limite: {self.limite}
              """

    def __str__(self) -> str:
        return self.imprimir_datos(0)

In [325]:
class operacion_miniconsultas_sql:
    """
        Clase que controla todos los datos necesarios para realizar
        una consulta que originalmente era una operacion de conjuntos,
        utilizando miniconsultas de complejidad menor

        atributos
        ------------
        operacion: Un string con el tipo de operacion que se quiere realizar

        parte_derecha: El ejecutor necesario para procesar la consulta que esta
                       del lado derecho de la operacion
        
        parte_izquierda: El ejecutor necesario para procesar la consulta que esta
                         del lado izquierdo de la operacion

        resultado: El resultado de la ejecucion de esta operación

        metodos
        -------------
        ejecutar: Funcion que realiza la operacion indicada en los datos
    """
    operacion: str
    parte_derecha: Union[miniconsulta_sql, join_miniconsultas_sql]
    parte_izquierda: Union[miniconsulta_sql, join_miniconsultas_sql, operacion_miniconsultas_sql]
    restultado: str

    def __init__(self, 
                 operacion:str, 
                 parte_derecha: Union[miniconsulta_sql, join_miniconsultas_sql], 
                 parte_izquierda:Union[miniconsulta_sql, join_miniconsultas_sql, operacion_miniconsultas_sql]):
        
        self.operacion = operacion
        self.parte_derecha = parte_derecha
        self.parte_izquierda = parte_izquierda
    
    def ejecutar(self):
        """
            Aqui es donde se hara la operacion usando los resultados
            de las miniconsultas.

            Ten en cuenta que este ejecutar debe ser llamado despues de 
            haber realizado todas las peticiones al LLM y las miniconsultas
            deben haber sido ejecutadas antes de ejecutar esta funcion
        """
        raise Exception("Por implementar!!!")


    def imprimir_datos(self, nivel: int) -> str:
        return f"""
{nivel*'    '}CONSULTA OPERACION
{(nivel + 1)*'    '}operacion: {self.operacion}
{(nivel + 1)*'    '}parte_derecha: 
{self.parte_derecha.imprimir_datos(nivel+2)}
{(nivel + 1)*'    '}parte_izquierda: 
{self.parte_izquierda.imprimir_datos(nivel+2)}
              """
    def __str__(self) -> str:
          return self.imprimir_datos(0)

In [326]:
def obtener_tablas(consulta_sql_ast: Expression) -> tuple[list[str], dict[str,str]]:
    """
        Dada un ast de una consulta SQL de postgres obtiene todas las tablas 
        de la consulta. Esta función tiene en cuenta el from y los joins. 
        Ademas tiene en cuenta los alias

        Parametros
        --------------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        --------------
            Una lista con el nombre de todas las tablas de la consulta.

            Un diccionario cuyos key son los alias de cada tabla y los 
            valores son el nombre original de la tabla
    """
    
    tablas: list[str] = []
    tablas_alias: dict[str, str] = {}
    
    if consulta_sql_ast.key != 'select':
        raise Exception('La consulta SQL necesita tener un "SELECT"')

    if consulta_sql_ast.args.get('from') == None:
        raise Exception('La consulta SQL necesita tener un "FORM"')

    # Obtenemos la tabla que esta en el from   
    elementos_a_revisar = [consulta_sql_ast.args['from']]

    # Si tiene joins tenemos en cuenta esas tablas
    if consulta_sql_ast.args.get('joins') != None:
        elementos_a_revisar += consulta_sql_ast.args['joins']
    
    # Conseguimos los nombres originales de las tablas y sus alias
    # si es que tienen 
    
    for elemento in elementos_a_revisar:
        if elemento.key == 'from' or elemento.key == 'join':
            
            nombre_tabla = elemento.this.this.this
            alias_tabla = elemento.this.alias
            
            if nombre_tabla not in tablas:
                tablas.append(nombre_tabla)

            if alias_tabla != '':
                tablas_alias[alias_tabla] = nombre_tabla
    
    if DEBUG: logging.info('Se obtuvieron todas las tablas y alias de tablas de la consulta')
    
    return tablas, tablas_alias

In [327]:
def obtener_proyecciones_agregaciones(consulta_sql_ast: Expression, 
                                     tablas: list[str], 
                                     tablas_alias: dict[str, str]) -> tuple[dict[str, list[column]], list[Expression]]:
    """
        Dada un ast de una consulta SQL de postgres obtiene todas las proyecciones y 
        funciones de agregación que hay en el SELECT de la consulta.

        Tenga en cuenta que si la consulta tiene un * como unica proyeccion
        la funcion lanzara un error.

        Parametros
        --------------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL
        
        tabla: Una lista con el nombre de todas las tablas de la consulta

        tablas_alias: Un diccionario cuyos key son los alias de cada tabla y los 
            valores son el nombre original de la tabla 

        Retorna
        --------------
            Una tupla cuya primera componente es un diccionario cuyas key son la tabla 
            (o alias de tablas) con la que esta relacionado una o varias proyecciones en
            el select. Y los valores son una lista de columnas de la tabla. En la 
            segunda componente se tiene un a lista con las funciones de agregacion
            que estan en el select.
    """
    proyecciones: dict[str, list[column]] = {}
    agregaciones = []

    # Revisamos si la unica proyeccion es un *
    if (len(consulta_sql_ast.args['expressions']) == 1 and 
        consulta_sql_ast.args['expressions'][0].key == 'star'):
        raise Exception('Debe haber al menos una proyección en el Select')
    
    for elemento_a_revisar in consulta_sql_ast.args['expressions']:

        if elemento_a_revisar.key in FUNCIONES_AGREGACION:
            parametro_de_agregacion = elemento_a_revisar.this

            if (parametro_de_agregacion.key == 'column' and
                parametro_de_agregacion.table not in tablas and 
                tablas_alias.get(parametro_de_agregacion.table) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{parametro_de_agregacion.table}"')
            
            agregaciones.append(elemento_a_revisar)
        
        elif elemento_a_revisar.key == 'column':
            if (elemento_a_revisar.table not in tablas and 
                tablas_alias.get(elemento_a_revisar.table) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{elemento_a_revisar.table}"')
            
            if proyecciones.get(elemento_a_revisar.table) == None:
                proyecciones[elemento_a_revisar.table] = []

            proyecciones[elemento_a_revisar.table].append(elemento_a_revisar)
    
    if DEBUG: logging.info('Se obtuvieron proyecciones y funciones de agregación en el SELECT')

    return proyecciones, agregaciones

In [328]:
def obtener_tablas_condiciones(condicion: Expression) -> tuple[str, str]:
    """
    Dada una condicion obtiene cuales son las tablas implicadas en esa condicion.
    Puede darse el caso que hayan 0, 1 o 2 tablas en una condicion.

    Parametros
    --------------
    condicion: Una expresion de sqlglot que representa una condicion

    Retorna
    --------------
        Una tupla con las tablas implicadas en la condición    
    """

    tabla_izquierda = ''
    tabla_derecha = ''

    # caso en el que estamos trabajando con condiciones binarias >, <, <=, >=, LIKE
    if isinstance(condicion, Binary):

        nodo_izquierdo = condicion.this
        
        if nodo_izquierdo.key == 'column':
            tabla_izquierda = nodo_izquierdo.table
        
        nodo_derecho = condicion.args['expression']
        
        if nodo_derecho.key == 'column':
            tabla_derecha = nodo_derecho.table

    # caso en el que estamos trabajando con un NOT IN
    elif isinstance(condicion, Not) and isinstance(condicion.this,In):

        nodo_izquierdo = condicion.this.this
        
        if nodo_izquierdo.key == 'column':
            tabla_izquierda = nodo_izquierdo.table
    
    elif isinstance(condicion, In):

        nodo_izquierdo = condicion.this
        
        if nodo_izquierdo.key == 'column':
            tabla_izquierda = nodo_izquierdo.table
    
    if DEBUG: logging.info('Se obtuvo las tablas relacionadas con la condicion %s', condicion.sql())
    
    return tabla_izquierda, tabla_derecha

In [329]:
def obtener_condiciones(conector_inicial: Expression) -> list[Expression]:
    """
    Dado un conector de sqlglot (Casi siempre un AND) obtiene todas las 
    condiciones existentes en ese conector y todos los que esten contenidos
    en el.

    Parametros
    --------------
    conector_inicial: Un conector de sqlglot (Casi siempre un AND)

    Retorna
    --------------
    Una lista con todas condiciones dentro del contector  
    """
    conectores = [conector_inicial]
    condiciones = []

    # Pasamos recursivamente por todos los operadores del WHERE
    # Y obtenemos todas las condiciones
    while conectores != []:
        conector_actual = conectores.pop(0)
        
        # Caso base
        if conector_actual.key != 'and':
            condiciones.append(conector_actual)
            break

        # Revisamos la parte izquierda del and
        if conector_actual.this.key != 'and':
            condiciones.append(conector_actual.this)
        else:
            conectores.append(conector_actual.this)
        
        # Agregamos la parte derecha del and
        if conector_actual.args['expression'].key != 'and':
            condiciones.append(conector_actual.args['expression'])
    
    return condiciones

In [330]:
def obtener_condiciones_where(consulta_sql_ast: Expression) -> dict[list[Expression], dict[str, list[Expression]]]:
    """
        Dada un ast de una consulta SQL de postgres obtiene todas las condiciones
        del WHERE de la consulta

        Tenga en cuenta que esta funcion espera que en el WHERE solo hayan operadores
        AND.

        Si hay condiciones de OR deben estar entre parentesis, los conectores principales
        deben ser ANDs

        Parametros
        --------------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        --------------
            Una lista con todas las condiciones del WHERE
    """

    # Obtenemos todas las condiciones
    # Ten en cuenta que el and asocia a izquierda esta vez
    if consulta_sql_ast.args.get('where') == None:
         raise Exception(f'La consulta debe tener un WHERE. la consulta es: \n {consulta_sql_ast.sql()}')

    condiciones = obtener_condiciones(consulta_sql_ast.args['where'].this)

    condiciones_or = []

    for i in range(len(condiciones)):
          if condiciones[i].key == 'or' or (condiciones[i].key == 'paren' and condiciones[i].this.key == 'or'):
               condiciones_or.append(condiciones.pop(i))

    if DEBUG: logging.info('Se obtuvieron todas las condiciones del WHERE en la consulta')

    return {'condiciones': condiciones, 'condiciones or': condiciones_or}

In [331]:
def obtener_condiciones_having(consulta_sql_ast: Expression) -> dict[str, list[Expression]]:
     """
        Dada un ast de una consulta SQL de postgres obtiene todas las condiciones
        del HAVING de la consulta

        Tenga en cuenta que esta funcion espera que en el HAVING solo hayan operadores
        AND.

        Si hay condiciones de OR deben estar entre parentesis, los conectores principales
        deben ser ANDs

        Parametros
        --------------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        --------------
            Una lista con todas las condiciones del HAVING
    """
     
     # Obtenemos todas las condiciones
     # Ten en cuenta que el and asocia a izquierda esta vez
     if consulta_sql_ast.args.get('having') == None:
          raise Exception('La consulta debe tener un HAVING')

     condiciones = obtener_condiciones(consulta_sql_ast.args['having'].this)

     condiciones_or = []

     for i in range(len(condiciones)):
          if condiciones[i].key == 'or':
               condiciones_or.append(condiciones.pop(i))

     if DEBUG: logging.info('Se obtuvieron todas condiciones del HAVING de la consulta')

     return {'condiciones': condiciones, 'condiciones or': condiciones_or}   

In [332]:
def clasificar_condiciones_where(condiciones: list[Expression],
                           tablas: list[str], 
                           tablas_alias: dict[str, str]) -> dict[str, list[Expression]]:
    """
        Toma una lista de condiciones, una lista de tablas y un diccionario con los
        alises de las tablas. Clasifica cada condicion dependiendo de la tabla 
        a la que haga referencia        

        Parametros
        --------------
        condiciones: Una lista de condiciones

        tabla: Una lista con el nombre de todas las tablas de la consulta

        tablas_alias: Un diccionario cuyos key son los alias de cada tabla y los 
            valores son el nombre original de la tabla 

        Retorna
        --------------
            Un diccionario cuyas key son la tabla (o alias de tablas) con la que 
            esta relacionado una o varias condicones en el WHERE. Y los valores
            son una lista de dichas condiciones.
    """
        
    # Si una condicion depende de dos tablas lo clasificaremos con la tabla de la 
    # izquierda
    condiciones_por_tablas = {}
    for condicion in condiciones:
        tabla_izquierda, tabla_derecha = obtener_tablas_condiciones(condicion)
        
        if tabla_derecha == '' and tabla_izquierda == '':
            raise Exception(f'La condicion {condicion} no es valida')
        
        # verificamos que las tablas del lado izquierdo y derecho existan
        if (tabla_izquierda != '' and
            tabla_izquierda not in tablas and 
            tablas_alias.get(tabla_izquierda) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{tabla_izquierda}"')

        if (tabla_derecha != '' and
            tabla_derecha not in tablas and 
            tablas_alias.get(tabla_derecha) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{tabla_derecha}"')
    
        if tabla_izquierda != '':
            if condiciones_por_tablas.get(tabla_izquierda) == None:
                condiciones_por_tablas[tabla_izquierda] = []
            
            condiciones_por_tablas[tabla_izquierda].append(condicion)
            continue
        
        if tabla_derecha != '':
            if condiciones_por_tablas.get(tabla_derecha) == None:
                condiciones_por_tablas[tabla_derecha] = []
            
            condiciones_por_tablas[tabla_derecha].append(condicion)
            continue 
        
    if DEBUG: logging.info('Se clasificaron las condiciones de la consulta')
    
    return condiciones_por_tablas

In [333]:
def obtener_condiciones_joins(consulta_sql_ast: Expression, 
                              tablas: list[str], 
                              tablas_alias: dict[str, str],
                              condiciones:dict[str, Expression])-> dict[str, list[Expression]]:
    """
        Dada un ast de una consulta SQL de postgres obtiene todas las condiciones
        de los distintos JOINs

        Esta función solo tiene en cuenta las condiciones que 
        son de igualdad

        Tenga en cuenta que esta funcion tiene en cuenta el numero de 
        condiciones del WHERE que este relacionado a una tabla para 
        saber a que tabla debe asignar la condicion del JOIN. 

        Esta funcion asigna la condicion del JOIN a la tabla que tenga
        menos condiciones (contando las condiciones del WHERE o del JOIN si 
        ya se el asigno alguna)

        TODO:
            - Manejar los casos donde las condiciones no son de igualdad

        Parametros
        --------------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL
        
        tabla: Una lista con el nombre de todas las tablas de la consulta

        tablas_alias: Un diccionario cuyos key son los alias de cada tabla y los 
            valores son el nombre original de la tabla 

        condiciones: Un diccionario cuyas key son la tabla (o alias de tabla) con la que 
                     esta relacionado una o varias condicones en el WHERE. Y los valores
                     son una lista de dichas condiciones.

        Retorna
        --------------
            Un diccionario cuyas key son la tabla (o alias de tablas) con la que 
            esta relacionado una o varias condiciones en los JOINs. Y los valores
            son una lista de dichas condiciones.
    """
    
    elementos_a_revisar = []

    if consulta_sql_ast.args.get('joins') != None:
        elementos_a_revisar = consulta_sql_ast.args['joins']
    
    condiciones_joins = []
    for elemento in elementos_a_revisar:
        if elemento.args.get('on') == None:
            raise Exception('Todo JOIN debe tener un ON')
        
        condiciones_joins.append(elemento.args['on'])
    
    condiciones_por_tablas = {}
    
    for condicion in condiciones_joins:
        for nodo in [condicion.this, condicion.args['expression']]:
            if nodo.key != 'column':
                raise Exception(f'La condicion de JOIN {condicion} debe involucrar dos tablas')
                        
            if nodo.table == '':
                raise Exception(f'La condicion {condicion} no es valida')
            
            if (nodo.table not in tablas and 
                tablas_alias.get(nodo.table) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{nodo.table}"')

        # Calculamos cuantas condiciones estan relacionada con cada una de las tablas
        # que estan involucaradas en la condicion        
        
        numero_condiciones_tabla_izquierda = 0
        if condiciones.get(condicion.this.table) != None:
            numero_condiciones_tabla_izquierda += len(condiciones[condicion.this.table])
            
        if condiciones_por_tablas.get(condicion.this.table) != None:
            numero_condiciones_tabla_izquierda += len(condiciones_por_tablas[condicion.this.table])

        numero_condiciones_tabla_derecha = 0
        if condiciones.get(condicion.args['expression'].table) != None:
            numero_condiciones_tabla_derecha += len(condiciones[condicion.args['expression'].table])
        
        if condiciones_por_tablas.get(condicion.args['expression'].table) != None:
            numero_condiciones_tabla_derecha += len(condiciones_por_tablas[condicion.args['expression'].table])

        # Le añadimos la condicion a la tabla que tenga menos condiciones, para asi 
        # acotar mas el dominio de la consulta

        if numero_condiciones_tabla_izquierda < numero_condiciones_tabla_derecha:
            if condiciones_por_tablas.get(condicion.this.table) == None:
                condiciones_por_tablas[condicion.this.table] = []

            condiciones_por_tablas[condicion.this.table].append(condicion)
        else:
            if condiciones_por_tablas.get(condicion.args['expression'].table) == None:
                condiciones_por_tablas[condicion.args['expression'].table] = []

            condiciones_por_tablas[condicion.args['expression'].table].append(condicion)
    if DEBUG: logging.info('Se obtuvieron todas las condiciones en los ON de los JOIN en la consulta')
    return condiciones_por_tablas

In [334]:
def obtener_tablas_joins(consulta_sql_ast: Expression, 
                               tablas: list[str], 
                               tablas_alias: dict[str, str]) -> dict[str, list[Expression]]:
    """
        Dada un ast de una consulta SQL de postgres obtiene todas las condiciones
        de los distintos JOINs y devuelve las columnas de las tablas utilizadas
        en alguna de estas condiciones

        Parametros
        ------------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL
        
        tabla: Una lista con el nombre de todas las tablas de la consulta

        tablas_alias: Un diccionario cuyos key son los alias de cada tabla y los 
                      valores son el nombre original de la tabla 

        Retorna
        -----------

        Un diccionario cuya key son las distintas tablas utilizadas en alguna 
        condicion de un JOIN. Y sus valores son una lista con las distintas 
        columnas de dicha tabla los cuales fueron utilizados en alguna condición
        de JOIN
    """
    elementos_a_revisar = []

    if consulta_sql_ast.args.get('joins') != None:
        elementos_a_revisar = consulta_sql_ast.args['joins']
    
    condiciones_joins = []
    for elemento in elementos_a_revisar:
        if elemento.args.get('on') == None:
            raise Exception('Todo JOIN debe tener un ON')
        
        condiciones_joins.append(elemento.args['on'])
    
    proyecciones_por_tablas = {}
    
    for condicion in condiciones_joins:
        for nodo in [condicion.this, condicion.args['expression']]:
            if nodo.key != 'column':
                raise Exception(f'La condicion de JOIN {condicion} debe involucrar dos tablas')
                        
            if nodo.table == '':
                raise Exception(f'La condicion {condicion} no es valida')
            
            if (nodo.table not in tablas and 
                tablas_alias.get(nodo.table) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{nodo.table}"')
            
            tabla = nodo.table

            if proyecciones_por_tablas.get(tabla) == None:
                proyecciones_por_tablas[tabla] = []
            
            if nodo not in proyecciones_por_tablas[tabla]:
                proyecciones_por_tablas[tabla].append(nodo)
    
    if DEBUG: logging.info('Se repartieron las condiciones de JOINS para las tablas')
    
    return proyecciones_por_tablas

In [335]:
def obtener_dependencia(tabla:str, condiciones: list[Expression]):
    """
        Dada una tabla y una lista de condiciones revisa si existe 
        alguna condicion donde se relacione a esta tabla con otra. Lo
        que quiere decir que la primera tabla depende de otra.

        Tenga en cuenta que esta funcion supone que una tabla X solo
        puede depender de otra tabla Y. No es posible (por el momento)
        que X depende de Y y de otra tabla Z al mismo tiempo.

        TODO
        -----------------
        Modificar esta funcion para que maneje el caso de que una tabla
        X pueda depender de una tabla Y y otra Z

        Parametros
        ------------
        tabla: Un string que el alias (o nombre original) de la tabla la cual 
               se quiere verificar si depende de otra.
        
        condiciones: Una lista de expresiones de sqlglot. Estas expresiones 
                     representan condiciones
        
        Retorna
        ---------

        Un string vacio si la tabla no depende de ninguna otra tabla. O Un string
        con el nombre de la tabla de la que depende.
    """
    dependencia = ''
    for condicion in condiciones:
        for nodo in [condicion.this, condicion.args['expression']]:
            if nodo.key == 'column' and nodo.table != tabla:
                dependencia = nodo.table
            
    return dependencia

In [336]:
def obtener_group_by(consulta_sql_ast:Expression) -> list[dict[str, str]]:
    """
    Dada un ast de una consulta SQL de postgres el cual tiene uno GROUP BY
    obtiene todas las columnas necesarias para realizar la agrupacion

    Parametros
    -----------------

    consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                      ast de una consulta SQL
    
    Retorna
    ------------
    
    Una lista de diccionarios que almacenan las tablas y columnas necesarias 
    para realizar la agrupacion
    """
    if consulta_sql_ast.args.get('group') == None:
        raise Exception('La consulta SQL debe tener un group by')
    
    group_by_ast = consulta_sql_ast.args['group']

    if DEBUG: logging.info('Se obtuvo el GROUP BY de la consulta')

    return [ {'tabla': i.args['table'].this, 'columna': i.args['this'].this} for i in group_by_ast.args['expressions']]

In [337]:
def obtener_order_by(consulta_sql_ast: Expression)-> list[dict[str, str]]:
    """
        Dada una consulta QSL, extrae la instruccion ORDER BY que indica el 
        orden solicitado para mostrar los registros a devolver en la consulta.

        parametros
        -----------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL
        
        retorna
        --------
            Una lista de diccionarios que indican las propiedades del orden
            solicitado
    """
    if consulta_sql_ast.args.get('order') == None:
        raise Exception('La consulta SQL debe tener un order by')
    
    ordenes: list[sqlglot.expressions.Ordered] = consulta_sql_ast.args.get('order').args.get('expressions')

    if DEBUG: logging.info('Se obtuvo el ORDER BY de la consulta')

    return [ {'tabla': i.this.table, 
              'columna': i.this.this.this,
              'tipo': "DESC" if (i.args.get('desc') or i.args.get('desc') is None) else "ASC" } for i in ordenes if i is not None ]

In [338]:
def obtener_limit(consulta_sql_ast: Expression) -> int:
    """
        Dada una consulta QSL, extrae la instruccion que indica el limite
        de registros a devolver en la consulta.

        parametros
        -----------
        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL
        
        retorna
        --------
            Una entero que indica el limite
    """

    if DEBUG: logging.info('Se obtuvo el limit de la consulta')

    return int(str(consulta_sql_ast.args.get('limit').args.get('expression')))

In [339]:
def dividir_joins(consulta_sql_ast: Expression) -> dict[str, dict[str, Any]]:
    """
        Dada un ast de una consulta SQL de postgres el cual tiene cero o mas joins   
        obtiene la informacion suficiente para crear una o mas consultas con 
        complejidad igual o menor.

        Parametros
        -----------------

        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        ------------

        Un diccionario cuya claves son el alias (o nombre) de una tabla y los valores
        son otros diccionarios cuya claves son el nombre de la informacion de esa tabla
        y los valores son la informacion necesaria.
    """
        
    tablas, tablas_alias = obtener_tablas(consulta_sql_ast)
    
    proyecciones, agregaciones = obtener_proyecciones_agregaciones(consulta_sql_ast, tablas, tablas_alias)

    # Pasamos por todas las funciones de agregación y si trabajan sobre una columna de alguna tabla
    # la agregamos a las proyecciones

    for agregacion in agregaciones:
        if agregacion.this.key == "column":
            tabla = agregacion.this.args['table'].this

            logging.info(f"tabla: {tabla}, tablas: {str(tablas)}, tablas_alias: {str(tablas_alias)}, tablas_alias.get(tabla): {tablas_alias.get(tabla)}")
            if (tabla not in tablas and 
                tablas_alias.get(tabla) == None):
                raise Exception(f'No existe la tabla o alias de tabla "{tabla}"')

            if proyecciones.get(tabla) == None:
                proyecciones[tabla] = []

            proyecciones[tabla].append(agregacion.this) 
    
    condiciones, condiciones_or = obtener_condiciones_where(consulta_sql_ast).values()
    
    condiciones_having = []
    
    condiciones_having_or = []
    
    if consulta_sql_ast.args.get('having') != None:
        condiciones_having, condiciones_having_or = obtener_condiciones_having(consulta_sql_ast).values()

    condiciones_por_tablas = clasificar_condiciones_where(condiciones, tablas, tablas_alias)

    condiciones_joins_por_tablas = obtener_condiciones_joins(consulta_sql_ast, tablas, tablas_alias, condiciones_por_tablas)
    
    proyecciones_joins = obtener_tablas_joins(consulta_sql_ast, tablas, tablas_alias)

    aliases = tablas_alias.keys()

    datos_miniconsultas = {}
    for alias in aliases:
        datos_miniconsultas[alias] = {'tabla': tablas_alias[alias]}
        
        if proyecciones.get(alias) != None:
            datos_miniconsultas[alias]['proyecciones'] = proyecciones[alias]
        else: 
            datos_miniconsultas[alias]['proyecciones'] = []
        
        if proyecciones_joins.get(alias) != None:
            datos_miniconsultas[alias]['proyecciones'] += proyecciones_joins[alias]
        
        if condiciones_por_tablas.get(alias) != None:
            datos_miniconsultas[alias]['condiciones'] = condiciones_por_tablas[alias]
        else:
            datos_miniconsultas[alias]['condiciones'] = []

        if condiciones_joins_por_tablas.get(alias) != None:
            datos_miniconsultas[alias]['condiciones_joins'] = condiciones_joins_por_tablas[alias]
        else:
            datos_miniconsultas[alias]['condiciones_joins'] = []

    resultado = {'datos miniconsultas': datos_miniconsultas, 
                 'datos globales': {'agregaciones': agregaciones,
                                    'order by': [],
                                    'limite': -1,
                                    'group by': [],
                                    'condiciones or': condiciones_or,
                                    'condiciones having': condiciones_having, 
                                    'condiciones having or': condiciones_having_or}}
    
    if consulta_sql_ast.args.get('order') != None:
        resultado['datos globales']['order by'] = obtener_order_by(consulta_sql_ast)
    
    if consulta_sql_ast.args.get('group') != None:
        resultado['datos globales']['group by'] = obtener_group_by(consulta_sql_ast)

    if consulta_sql_ast.args.get('limit') != None:
        resultado['datos globales']['limite'] = obtener_limit(consulta_sql_ast)

    if DEBUG: logging.info('Se termino de dividir la consulta')

    return resultado

In [340]:
def obtener_miniconsultas_join(consulta_sql_ast: Expression) -> dict[str, list[miniconsulta_sql]]:
    """
        Dada un ast de una consulta SQL de postgres con cero o mas joins lo divide en consultas mas 
        simples de forma tal que despues se pueda usar la informacion de estas 
        nuevas consultas mas pequeñas para hacerle preguntas a algun LLM.

        Parametros
        -----------------

        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        ------------

        Un diccionario con las miniconsultas que son dependientes (necesitan del 
        resultado de otra miniconsulta) y las independientes.
    """
    if DEBUG: logging.info('La consulta tiene 0 o mas JOINS, se procede a dividir la consulta')


    datos_divididos_joins = dividir_joins(consulta_sql_ast)
    datos_miniconsultas = datos_divididos_joins['datos miniconsultas']
    datos_globales = datos_divididos_joins['datos globales']
    dependencias = {}
    
    aliases = datos_miniconsultas.keys()
    
    miniconsultas_independientes = {}
    miniconsultas_dependientes = {}
    
    for alias in aliases:
        dependencia = obtener_dependencia(alias, datos_miniconsultas[alias]['condiciones_joins'])

        if dependencia != '':
            dependencias[alias] = dependencia
            miniconsultas_dependientes[alias] = miniconsulta_sql(tabla = datos_miniconsultas[alias]['tabla'], 
                                                                 alias = alias,
                                                                 proyecciones = datos_miniconsultas[alias]['proyecciones'], 
                                                                 condiciones = datos_miniconsultas[alias]['condiciones'],
                                                                 condiciones_join = datos_miniconsultas[alias]['condiciones_joins'])
        else:
            miniconsultas_independientes[alias] = miniconsulta_sql(tabla = datos_miniconsultas[alias]['tabla'], 
                                                                  alias = alias,
                                                                  proyecciones = datos_miniconsultas[alias]['proyecciones'], 
                                                                  condiciones = datos_miniconsultas[alias]['condiciones'],
                                                                  condiciones_join = datos_miniconsultas[alias]['condiciones_joins'])
       
    for alias, dependencia in dependencias.items():
        if miniconsultas_independientes.get(dependencia) != None:
            dependencia = miniconsultas_independientes[dependencia]
        else:
            dependencia = miniconsultas_dependientes[dependencia]
        
        miniconsultas_dependientes[alias].dependencia = dependencia
    
    lista_condiciones_join = []
    for miniconsulta in list(miniconsultas_dependientes.values()) + list(miniconsultas_independientes.values()):
        lista_condiciones_join += miniconsulta.condiciones_join

    return {'ejecutor': join_miniconsultas_sql(condiciones_join = lista_condiciones_join,
                                               miniconsultas_dependientes = list(miniconsultas_dependientes.values()), 
                                               miniconsultas_independientes = list(miniconsultas_independientes.values()),
                                               lista_group_by = datos_globales['group by'],
                                               lista_order_by = datos_globales['order by'],
                                               limite = datos_globales['limite'],
                                               condiciones_or = datos_globales['condiciones or'],
                                               condiciones_having = datos_globales['condiciones having'],
                                               condiciones_having_or = datos_globales['condiciones having or'],
                                               lista_agregaciones = datos_globales['agregaciones']),
            'dependientes': list(miniconsultas_dependientes.values()), 
            'independientes': list(miniconsultas_independientes.values())}

In [341]:
def obtener_miniconsultas_operacion(consulta_sql_ast:Expression) -> dict[str, list[miniconsulta_sql]]:
    """
        Dada un ast de una consulta SQL de postgres con una o mas operaciones de conjuntos, lo divide  
        en consultas mas simples de forma tal que despues se pueda usar la  
        informacion de estas nuevas consultas mas pequeñas para hacerle preguntas 
        a algun LLM.

        Ten en cuenta que sqlglot asocia a 'izquierda' o mejor dicho asocia 
        hacia arriba
        
        Parametros
        -----------------

        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        ------------

        Un diccionario con las miniconsultas que son dependientes (necesitan del 
        resultado de otra miniconsulta) y las independientes.
    """
    if consulta_sql_ast.key not in OPERACIONES_CONJUNTOS:
        raise Exception("Para ejecutar esta funcion la consulta SQL debe tener al menos una operacion")
    
    # la parte derecha de una operacion siempre sera una consulta que no es una operacion
    if DEBUG: logging.info('La consulta es una operación de conjuntos, se procede a parsear las consultas relacionada con esta operacion')

    miniconsultas_derecha =  obtener_miniconsultas(consulta_sql_ast.args['expression'].sql())

    miniconsultas_izquierda = obtener_miniconsultas(consulta_sql_ast.this.sql())

    miniconsultas_totales = {'dependientes': miniconsultas_izquierda['dependientes'] + miniconsultas_derecha['dependientes'], 
                             'independientes': miniconsultas_izquierda['independientes'] + miniconsultas_derecha['independientes']}

    miniconsultas_totales['ejecutor'] = operacion_miniconsultas_sql(consulta_sql_ast.key, miniconsultas_derecha['ejecutor'], miniconsultas_izquierda['ejecutor'])
    return miniconsultas_totales

In [342]:
def obtener_miniconsultas(consulta_sql: str)-> dict[str, list[miniconsulta_sql]]:
    """
        Divide una consulta SQL en miniconsultas de menor complejidad
        y devuelve una lista con las distintas miniconsultas a ejecutar

        parametros
        -----------
        consulta_sql: Un string con la consulta SQL en sintaxis de Postgres

        retorna
        --------
            Una lista con las distintas miniconsultas a ejecutar
    """
    miniconsultas = {}
    consulta_sql_ast = parse_one(consulta_sql, dialect='postgres')

    # Caso donde la consulta es una operacion de conjuntos
    if consulta_sql_ast.key in OPERACIONES_CONJUNTOS:
        return obtener_miniconsultas_operacion(consulta_sql_ast)

    condiciones = obtener_condiciones_where(consulta_sql_ast)['condiciones']

    # Caso donde la consulta es una consulta anidada
    for condicion in condiciones:
        if (isinstance(condicion, In) or
            isinstance(condicion, Not) or
            isinstance(condicion, Binary) and isinstance(condicion.this, Subquery) or 
            isinstance(condicion, Binary) and isinstance(condicion.args.get('expression'), Subquery)):
            return obtener_miniconsultas_anidadas(consulta_sql_ast)
        
#           
    # Caso donde la consulta es un select sin condicion IN
    if consulta_sql_ast.key == 'select':
        return obtener_miniconsultas_join(consulta_sql_ast)

    return miniconsultas    

In [343]:
def obtener_lista_miniconsultas(consulta_sql: str) -> list[miniconsulta_sql]:
    """
        Dado una consulta SQL lo divide en consultas mas simples de forma tal
        que despues se pueda usar la informacion de estas nuevas consultas
        mas pequeñas para hacerle preguntas a algun LLM.

        Tenga en cuenta que el resultado de esta funcion es una lista donde 
        las primeras consultas son consultas que dependen de otras, y las 
        ultimas son consultas que no depende de ninguna otra.

        Parametros
        -----------------

        consulta_sql: Un string con la consulta SQL

        Retorna
        ------------

        Una lista con las distintas consultas mas simples a realizar para devolver
        la información que requiere la consulta original.
    """
    miniconsultas = obtener_miniconsultas(consulta_sql)
    
    return miniconsultas['dependientes'] + miniconsultas['independientes']

In [344]:
def obtener_ejecutor(consulta_sql:str):
    """
        Divide una consulta SQL en miniconsultas de menor complejidad
        y devuelve el ejecutor necesario para combinar las miniconsultas
        de forma tal que se obtenga un resulado suficiente para responder
        la consulta SQL original

        parametros
        -----------
        consulta_sql: Un string con la consulta SQL en sintaxis de Postgres

        retorna
        --------
            El ejecutor necesario para combinar las miniconsultas
    """
    return obtener_miniconsultas(consulta_sql)['ejecutor']

In [345]:
class miniconsulta_sql_anidadas:
    """
        Clase que controla todos los datos necesarios para realizar
        una consulta que originalmente era una consulta anidada, utilizando
        miniconsultas de complejidad menor

        atributos
        ------------        
        proyecciones: Lista con todas las proyecciones en el SELECT

        agregaciones: Lista con todas las agregaciones en el SELECT

        aliases: Lista con todos los alias o nombre de Tablas utilizadas
                 en la consulta
        
        tablas_aliases: Diccionario con el nombre de los aliases utilizados
        
        condiciones_having_or: Lista con todas las disyunciones que existen
                               en el HAVING del JOIN

        lista_agregaciones: Lista con todos las funciones de agregación que 
                            esta en el SELECT del JOIN
        
        lista_group_by: Lista con todas las columnas del GROUP BY

        lista_order_by: Lista con todas las columnas del ORDER BY

        condiciones_join: Una lista con las distintas condiciones
                          utilizadas en los joins de la consulta
                          SQL original
        
        condiciones_or: Lista con todas las disyunciones que existen
                        en el WHERE del JOIN
        
        condiciones_having: Las condiciones del HAVING que no son disyunciones
        
        resultado: El resultado de la ejecucion de este join

        limite: Un entero que indica si el JOIN tiene un LIMIT o no (Si tiene
                un -1 quiere decir que no tienen LIMIT)
        
        subconsultas: Una lista de diccionarios con toda la información sobre 
                      todas las condiciones anidadas dentro de la consulta
    """

    proyecciones: list[Expression]
    agregaciones: list[Expression]
    aliases: list[str]
    tablas_aliases: dict[str, str]
    condiciones_join: list[Expression]
    condiciones: list[Expression]
    condiciones_or: list[Expression]
    condiciones_having: list[Expression]
    condiciones_having_or: list[Expression]
    limite: int
    resultado: str
    lista_order_by: list[str]
    lista_group_by: list[dict[str,str]]
    subconsultas: list[dict[str, str | Expression]]

    def __init__(self, 
                 proyecciones: list[Expression],
                 agregaciones: list[Expression],
                 aliases: list[str],
                 tablas_aliases: dict[str, str],
                 condiciones: list[Expression],
                 condiciones_or: list[Expression],
                 condiciones_having: list[Expression],
                 condiciones_having_or: list[Expression],
                 condiciones_join: list[Expression] = [],
                 limite: int = -1,
                 lista_order_by: list[str] = [],
                 lista_group_by: list[dict[str,str]] = []) -> None:
        self.proyecciones = proyecciones
        self.agregaciones = agregaciones
        self.aliases = aliases
        self.tablas_aliases = tablas_aliases
        self.condiciones = condiciones
        self.condiciones_or = condiciones_or
        self.condiciones_having =  condiciones_having
        self.condiciones_having_or =  condiciones_having_or
        self.condiciones_join = condiciones_join
        self.limite = limite
        self.lista_order_by = lista_order_by
        self.lista_group_by = lista_group_by
        self.subconsultas, self.condiciones = self.__obtener_subconsultas(condiciones)
            
    def __construir_miniconsulta(self, 
                                 consulta_sql_ast: Expression) -> (miniconsulta_sql 
                                                                   | join_miniconsultas_sql):
        raise Exception("Por implementar!!!")
    
    def __obtener_subconsultas(self,
                               condiciones: list[Expression]) -> tuple[list[dict[str, str | Expression]], list[Expression]]:
    

        consultas: list[dict[str, str | Expression]] = []
        indices: list[int] = []
    
        for i, cond in enumerate(condiciones):
            if (isinstance(cond, Binary)): 
                if (isinstance(cond.args.get('this'), Subquery)): 
                    indices.append(i)
                    consultas.append({'operacion': cond.key,
                                'tabla': cond.args.get('expression').args.get('table').args.get('this'),
                                'columna': cond.args.get('expression').args.get('this').args.get('this'),
                                'subquery': obtener_ejecutor(cond.args.get('this').args.get('this').sql())})

                elif (isinstance(cond.args.get('expression'), Subquery)):
                    indices.append(i)
                    consultas.append({'operacion': cond.key,
                                'tabla': cond.args.get('this').args.get('table').args.get('this'),
                                'columna': cond.args.get('this').args.get('this').args.get('this'),
                                'subquery': obtener_ejecutor(cond.args.get('expression').args.get('this').sql())})
            
            elif (isinstance(cond, Not)):
                indices.append(i)
                consultas.append({'operacion': f'{cond.key} {cond.args.get("this").key}',
                                'tabla': cond.args.get('this').args.get('this').args.get('table').args.get('this'),
                                'columna': cond.args.get('this').args.get('this').args.get('this').args.get('this'),
                                'subquery': obtener_ejecutor(cond.args.get('this').args.get('query').args.get('this').sql())})
                
            elif (isinstance(cond, In)):
                indices.append(i)
                consultas.append({'operacion': cond.key,
                            'tabla': cond.args.get('this').args.get('table').args.get('this'),
                            'columna': cond.args.get('this').args.get('this').args.get('this'),
                            'subquery': obtener_ejecutor(cond.args.get('query').args.get('this').sql())})

        return (consultas, [condiciones[i] for i in range(len(condiciones)) if i not in indices])

    def imprimir_datos(self, nivel: int) -> str:
        return f"""
{nivel*'    '}CONSULTA ANIDADA
{(nivel + 1)*'    '}proyecciones: {self.proyecciones}
{(nivel + 1)*'    '}agregaciones: {[i.sql() for i in self.agregaciones]}
{(nivel + 1)*'    '}aliases: {self.aliases}
{(nivel + 1)*'    '}tablas_aliases: {self.tablas_aliases}
{(nivel + 1)*'    '}condiciones_join: {[i.sql() for i in self.condiciones_join]}
{(nivel + 1)*'    '}condiciones: {[i.sql() for i in self.condiciones]}
{(nivel + 1)*'    '}condiciones_or: {[i.sql() for i in self.condiciones_or]}
{(nivel + 1)*'    '}condiciones_having: {self.condiciones_having}
{(nivel + 1)*'    '}condiciones_having_or: {[i.sql() for i in self.condiciones_having_or]}
{(nivel + 1)*'    '}limite: {self.limite}
{(nivel + 1)*'    '}lista_order_by: {self.lista_order_by}
{(nivel + 1)*'    '}lista_group_by: {self.lista_group_by}
{(nivel + 1)*'    '}subconsultas: {self.subconsultas}
{(nivel + 1)*'    '}"""

    def __str__(self) -> str:
        return self.imprimir_datos(0)

In [346]:
def obtener_miniconsultas_anidadas(consulta_sql_ast: Expression):
    """
        Dada un ast de una consulta SQL de postgres con una o mas condiciones anidadas lo  
        divide en consultas mas simples de forma tal que despues se pueda usar la informacion  
        de estas nuevas consultas mas pequeñas para hacerle preguntas a algun LLM.

        Parametros
        -----------------

        consulta_sql_ast: Un objeto Expression de sqlglot. Representa un 
                          ast de una consulta SQL

        Retorna
        ------------

    """

    aliases, tablas_alias = obtener_tablas(consulta_sql_ast)
    condiciones, condiciones_or = obtener_condiciones_where(consulta_sql_ast).values()
    proyecciones, agregaciones = obtener_proyecciones_agregaciones(consulta_sql_ast, aliases, tablas_alias)
    condiciones_clasificada = clasificar_condiciones_where(condiciones, aliases, tablas_alias)
    condiciones_joins = obtener_condiciones_joins(consulta_sql_ast, aliases, tablas_alias, condiciones_clasificada)
    
    limite = -1
    lista_order_by = []
    lista_group_by = []
    lista_having = []
    lista_having_or = []
    if consulta_sql_ast.args.get('order') != None:
        lista_order_by = obtener_order_by(consulta_sql_ast)
    
    if consulta_sql_ast.args.get('group') != None:
        lista_group_by = obtener_group_by(consulta_sql_ast)

    if consulta_sql_ast.args.get('limit') != None:
        limite = obtener_limit(consulta_sql_ast)

    if consulta_sql_ast.args.get('having') != None:
        lista_having, lista_having_or = obtener_condiciones_having(consulta_sql_ast).values()

    return {"ejecutor": miniconsulta_sql_anidadas(proyecciones = proyecciones, 
                                                  agregaciones = agregaciones, 
                                                  aliases = aliases, 
                                                  tablas_aliases = tablas_alias, 
                                                  condiciones = condiciones, 
                                                  condiciones_or = condiciones_or,
                                                  condiciones_join = condiciones_joins, 
                                                  limite = limite, 
                                                  lista_order_by = lista_order_by, 
                                                  lista_group_by = lista_group_by,
                                                  condiciones_having = lista_having,
                                                  condiciones_having_or = lista_having_or)}
    

In [347]:
# Zona de prueba

if DEBUG:
    nombre = 'log_parser_' + datetime.today().strftime('%d_%m_%Y') + ".log"

    logging.basicConfig(filename=nombre, 
                        filemode='w', 
                        format='%(levelname)s - %(message)s',
                        level=logging.INFO,
                        force=True)


# consulta_sql = """
#                 select distinct t3.name 
#                 from country as t1 
#                 join countrylanguage as t2 on t1.code = t2.countrycode 
#                 join city as t3 on t3.countrycode = t1.verga  
#                 where t2.isofficial = 't' and (t2.language = 'chinese' or t1.continent = "asia") and t3.nose = 1 and t3.otra = 2 and t3.jejox = 3
#             """

# consulta_sql = """
#                 select t1.name
#                 from country as t1 
#                 join countrylanguage as t2 on t1.code = t2.countrycode 
#                 join tumadre as t3 on t1.age = t3.age
#                 where t2.isofficial = 't' and t2.language = 'chinese' and t1.nose = panqueca
#                 """

# consulta_sql = """
                # SELECT t1.border 
                # FROM border_info as t1
                # WHERE t1.state_name IN ( SELECT t2.border 
                #       FROM border_info as t2
                #       WHERE t2.state_name = "colorado" 
                #     );
#                 """

consulta_sql = """
                select t1.a
                from nose as t1
                join hola as t2 on t1.c = t2.a
                where t1.b = 'x'
               """

# consulta_sql = """
#                 SELECT SUM(T2.Name)
#             FROM
#                 country AS T1
#                 JOIN city AS T2 ON T2.CountryCode = T1.Code
#             WHERE
#                 T1.Continent = 'Europe'
#                 AND T1.Name NOT IN (
#                     SELECT
#                         T3.Name
#                     FROM
#                         country AS T3
#                         JOIN countrylanguage AS T4 ON T3.Code = T4.CountryCode
#                     WHERE
#                         T4.IsOfficial = 'T'
#                         AND T4.Language = 'English'
#                 )
#                """

# print(parse_one(consulta_sql).sql)
# aliases, tablas_alias = obtener_tablas(parse_one(consulta_sql, dialect='postgres'))

# print(obtener_proyecciones(parse_one(consulta_sql, dialect='postgres'),aliases, tablas_alias))

# print(obtener_condiciones(parse_one(consulta_sql, dialect='postgres'),aliases, tablas_alias))

# condiciones = obtener_condiciones(parse_one(consulta_sql, dialect='postgres'))
# condiciones_por_tablas = clasificar_condiciones_where(condiciones, aliases, tablas_alias)
# print(condiciones_por_tablas)

# print(obtener_proyecciones_joins(parse_one(consulta_sql, dialect='postgres'),aliases, tablas_alias))

print(obtener_miniconsultas(consulta_sql))

# print(obtener_lista_miniconsultas(consulta_sql))


{'ejecutor': <__main__.join_miniconsultas_sql object at 0x73e411e47190>, 'dependientes': [
MINI CONSULTA
    tabla: hola
    alias: t2
    proyecciones: ['t2.a']
    condiciones: []
    condiciones_join: ['t1.c = t2.a']
    dependencia: [
        MINI CONSULTA
            tabla: nose
            alias: t1
            proyecciones: ['t1.a', 't1.c']
            condiciones: ["t1.b = 'x'"]
            condiciones_join: []
            dependencia: []
            status: En Espera
            ]
    status: En Espera
    ], 'independientes': [
MINI CONSULTA
    tabla: nose
    alias: t1
    proyecciones: ['t1.a', 't1.c']
    condiciones: ["t1.b = 'x'"]
    condiciones_join: []
    dependencia: []
    status: En Espera
    ]}


In [348]:
import pandas as pd

if DEBUG:
    nombre = 'log_parser_' + datetime.today().strftime('%d_%m_%Y') + ".log"

    logging.basicConfig(filename=nombre, 
                        filemode='w', 
                        format='%(levelname)s - %(message)s',
                        level=logging.INFO,
                        force=True)

df = pd.read_csv('ignorar/queries_ejecutar.csv', sep=";")
for i, fila in df.iterrows():
    if fila.loc['ejecutar'] == "No":
        continue

    logging.warning(f'parseando la consulta nro.{i + 1}')
    logging.warning(f'Esta es:  {fila.loc["query"]}')
    try:
        resultado = obtener_ejecutor(fila.loc['query'])
        logging.warning(f'Se parseo la consulta nro.{i + 1}')
        logging.warning(f'Esta es:  {fila.loc["query"]}')
        logging.warning(f'Se obtuvo el siguiente resultado:\n{str(resultado)}')
    except:
        logging.error(f'No se pudo completar el parse de la consulta {fila.loc["query"]}', exc_info=True)
